In [548]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 


In [549]:
file_to_open = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df = pd.read_html(file_to_open)[0]
df.head(5)


,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park / Ontario Provincial Government,M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned


In [550]:
# convert grid format to a set rows:
df = df.stack() 
#now we have a Pandas series(1D), which we convert back to a dataframe (2D)
df = df.to_frame()
df.columns = ['data']
df.head()

data
0 0                                  M1ANot assigned
  1                                  M2ANot assigned
  2                         M3ANorth York(Parkwoods)
  3                  M4ANorth York(Victoria Village)
  4  M5ADowntown Toronto(Regent Park / Harbourfront)

In [551]:
# Pandas messing about with index, lets reset 
df.reset_index(drop=True,inplace=True)

In [552]:
# each cell starts with a 3 digits postal code which we extract
df['PostalCode'] = df['data'].str[:3]
df['data'] = df['data'].str[3:]
df.head()

,data,PostalCode
0,Not assigned,M1A
1,Not assigned,M2A
2,North York(Parkwoods),M3A
3,North York(Victoria Village),M4A
4,Downtown Toronto(Regent Park / Harbourfront),M5A


In [553]:
# Extract the portion within the parentheses to a new colomn Neighborhood
df['Neighborhood'] = df['data'].str.extract('\((.*?)\)')
df

,data,PostalCode,Neighborhood
0,Not assigned,M1A,NaN
1,Not assigned,M2A,NaN
2,North York(Parkwoods),M3A,Parkwoods
3,North York(Victoria Village),M4A,Victoria Village
4,Downtown Toronto(Regent Park / Harbourfront),M5A,Regent Park / Harbourfront
5,North York(Lawrence Manor / Lawrence Heights),M6A,Lawrence Manor / Lawrence Heights
6,Queen's Park / Ontario Provincial Government,M7A,NaN
7,Not assigned,M8A,NaN
8,Etobicoke(Islington Avenue),M9A,Islington Avenue
9,Scarborough(Malvern / Rouge),M1B,Malvern / Rouge


In [554]:
# Remove the neighborhood so that only the borough remains
df['Borough'] = df['data'].str.split('(').str[0]
df.head()

,data,PostalCode,Neighborhood,Borough
0,Not assigned,M1A,NaN,Not assigned
1,Not assigned,M2A,NaN,Not assigned
2,North York(Parkwoods),M3A,Parkwoods,North York
3,North York(Victoria Village),M4A,Victoria Village,North York
4,Downtown Toronto(Regent Park / Harbourfront),M5A,Regent Park / Harbourfront,Downtown Toronto


In [555]:
# no longer necessary
df.drop(columns = ['data'], inplace=True)

In [556]:
# rearange columns
df = df[['PostalCode', 'Borough', 'Neighborhood']]
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [557]:
# replace / with , and rmove space
df['Neighborhood'] = df['Neighborhood'].str.replace(' /', ',')
df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Queen's Park / Ontario Provincial Government,NaN
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"


In [558]:
# Boolean mask to identify where Neighborhood field empty but borough is not
mask1 = ((df['Neighborhood'].isnull()) & (df['Borough'] != "Not assigned"))
mask1.head(10)

0    False
1    False
2    False
3    False
4    False
5    False
6     True
7    False
8    False
9    False
dtype: bool

In [559]:
# apply mask
df['Neighborhood'][mask1] = df['Borough']
df.head(7)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Queen's Park / Ontario Provincial Government,Queen's Park / Ontario Provincial Government


In [560]:
# remove postal codes that have no borough assigned 
df.drop(df[df.Borough =="Not assigned"].index, inplace=True)


In [561]:
# reset index numbering
df.index = np.arange(len(df))
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park / Ontario Provincial Government,Queen's Park / Ontario Provincial Government


In [562]:
df.shape

(103, 3)